# HeteroGAT Experiments

In [1]:
import os
import sys

import numpy as np
import pandas as pd

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch_geometric as pyg
from torch_geometric.nn import GATConv, HeteroConv, Linear
from torch.nn import functional as F

from src import temporal_loader_v2 as tl
from src.utils import score

In [2]:
cuda_device = 1
pyg.seed_everything(42)

### Model

In [3]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GATConv, HeteroConv, Linear


class HeteroGNN(torch.nn.Module):
    def __init__(self, metadata, hidden_channels, out_channels, num_layers):
        super().__init__()

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HeteroConv({
                edge_type: GATConv((-1,-1), hidden_channels)
                for edge_type in metadata[1]
            })
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
            x_dict = {key: F.leaky_relu(x) for key, x in x_dict.items()}
        return self.lin(x_dict['domain_node'])
    

def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)
    mask = data['domain_node'].train_mask
    loss = F.cross_entropy(out[mask], data['domain_node'].y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test(model, data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict).argmax(dim=-1)

    accs = []
    for split in ['train_mask', 'val_mask']:
        mask = data['domain_node'][split]
        acc = (pred[mask] == data['domain_node'].y[mask]).sum() / mask.sum()
        accs.append(float(acc))
    return accs

In [4]:
def experiment(start,end,test_list, model_type):
    kg_path = lambda graph_name: f'../data/{graph_name}'

    dataset = tl.DNS(root=kg_path('DNS_2m'), start=start, end=end, test_list=test_list, balance_gt=False, domain_file='domains2.csv')
    data = dataset.train_data # training data
    
    model = HeteroGNN(data.metadata(), hidden_channels=64, out_channels=2, num_layers=2)

    if torch.cuda.is_available():
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        torch.cuda.set_device(cuda_device)

        data, model = data.to(device), model.to(device)

    with torch.no_grad():  # Initialize lazy modules.
        out = model(data.x_dict, data.edge_index_dict)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)

    for epoch in range(0, 201):
        loss = train(model, data, optimizer)
        train_acc, val_acc = test(model,data)
        if epoch % 20 == 0:
            print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, '
                f'Val: {val_acc:.4f}')
        
    model.eval()
    for index, test_data in enumerate(dataset.test_data):
        test_data = test_data.to(device)
        with torch.no_grad():
            pred = model(test_data.x_dict, test_data.edge_index_dict).argmax(dim=-1)
        mask = test_data['domain_node']['val_mask']
        scores = score(pred[mask],test_data['domain_node'].y[mask])
        with open("../results_copy.csv", "a") as logger:
            logger.write("{},{},{},{},".format(model_type,start,end,index))
            logger.write(",".join(str(x) for x in scores.values()))
            logger.write('\n')


        for metric, val in scores.items():
            print(metric, ':{:.4f}'.format(val))
    
for model_type in ['heterogat']:
    for i in range(5):
        experiment(i,i+6,[i+7,i+8], model_type)

Total labeled 897635
Labeled node count for 0, 6: 31778
Labeled node count for 0, 7: 2610
Labeled node count for 0, 8: 2083
Epoch: 000, Loss: 0.6810, Train: 0.6810, Val: 0.6810
Epoch: 020, Loss: 0.4557, Train: 0.7881, Val: 0.7910
Epoch: 040, Loss: 0.4062, Train: 0.8088, Val: 0.8082
Epoch: 060, Loss: 0.3968, Train: 0.8101, Val: 0.8105
Epoch: 080, Loss: 0.3925, Train: 0.8122, Val: 0.8137
Epoch: 100, Loss: 0.3893, Train: 0.8135, Val: 0.8118
Epoch: 120, Loss: 0.3875, Train: 0.8146, Val: 0.8148
Epoch: 140, Loss: 0.3869, Train: 0.8140, Val: 0.8129
Epoch: 160, Loss: 0.3850, Train: 0.8150, Val: 0.8135
Epoch: 180, Loss: 0.3854, Train: 0.8153, Val: 0.8093
Epoch: 200, Loss: 0.3833, Train: 0.8142, Val: 0.8143
tn, fp, fn, tp 1339 266 162 843
acc :0.8360
f1 :0.8373
auc :0.8365
prec :0.7601
recall :0.8388
fpr :0.1657
mi_f1 :0.8360
ma_f1 :0.8299
tn, fp, fn, tp 1092 217 143 631
acc :0.8272
f1 :0.8286
auc :0.8247
prec :0.7441
recall :0.8152
fpr :0.1658
mi_f1 :0.8272
ma_f1 :0.8183
Total labeled 897635
La